In [1]:
import sqlite3
import pandas as pd
from prettytable import PrettyTable

In [2]:
conn = sqlite3.connect('test_restore_protdb.db')
cur = conn.cursor()

In [3]:
print('2.Species in the database:')
cursor = cur.execute('select * from species')
x = PrettyTable([headers[0] for headers in cursor.description])
for row in cursor.fetchall():
    x.add_row(row)

2.Species in the database:


In [4]:
print(x)

+--------+-------------------------+---------------------+
| abbrev |           name          |        common       |
+--------+-------------------------+---------------------+
|   Hs   |       Homo sapiens      |        Human        |
|   Mm   |       Mus musculus      |        Mouse        |
|   Gg   |      Gallus gallus      |       Chicken       |
|   Xl   |      Xenopus laevis     | African clawed frog |
|   Bt   |        Bos taurus       |         Cow         |
|   Tr   |    Takifugu rubripes    |      Pufferfish     |
|   Br   |    Brachydanio rerio    |      Zebrafish      |
|   Dm   | Drosophila melanogaster |         Fly         |
|   Ce   |  Caenorhabditis elegans |       Nematode      |
|   Ss   |        Sus scrofa       |      Wild Boar      |
+--------+-------------------------+---------------------+


In [5]:
print("3.Adding species 'Sus scrofa':")
cursor = cur.execute("INSERT OR IGNORE INTO species VALUES ('Ss', 'Sus scrofa', 'Wild Boar');")
cursor = cur.execute("COMMIT")
cursor = cur.execute('select * from species')
x = PrettyTable([headers[0] for headers in cursor.description])
for row in cursor.fetchall():
    x.add_row(row)

3.Adding species 'Sus scrofa':


In [6]:
print(x)

+--------+-------------------------+---------------------+
| abbrev |           name          |        common       |
+--------+-------------------------+---------------------+
|   Hs   |       Homo sapiens      |        Human        |
|   Mm   |       Mus musculus      |        Mouse        |
|   Gg   |      Gallus gallus      |       Chicken       |
|   Xl   |      Xenopus laevis     | African clawed frog |
|   Bt   |        Bos taurus       |         Cow         |
|   Tr   |    Takifugu rubripes    |      Pufferfish     |
|   Br   |    Brachydanio rerio    |      Zebrafish      |
|   Dm   | Drosophila melanogaster |         Fly         |
|   Ce   |  Caenorhabditis elegans |       Nematode      |
|   Ss   |        Sus scrofa       |      Wild Boar      |
+--------+-------------------------+---------------------+


In [7]:
#4.count the proteins with a sequence longer than 1000
cursor = cur.execute("SELECT accession FROM protein WHERE LENGTH(sequence)>1000;")
x = PrettyTable([headers[0] for headers in cursor.description])
for row in cursor.fetchall():
    x.add_row(row)
print(x)

+------------+
| accession  |
+------------+
| E75A_DROME |
| E75B_DROME |
| E75C_DROME |
| HR38_DROME |
| FTF1_DROME |
+------------+


In [8]:
#5.print species names matching NHR3 family
cursor = cur.execute('''
SELECT DISTINCT(name)
FROM species, familymembers, protein
WHERE protein.species = species.abbrev
AND familymembers.protein = protein.accession
AND familymembers.family = 'NHR3';
'''
)
x = PrettyTable([headers[0] for headers in cursor.description])
for row in cursor.fetchall():
    x.add_row(row)
print(x)

+-------------------+
|        name       |
+-------------------+
|    Homo sapiens   |
|    Mus musculus   |
| Brachydanio rerio |
|   Gallus gallus   |
|   Xenopus laevis  |
|     Bos taurus    |
+-------------------+


In [9]:
#5.print species names matching NHR3 family
cursor = cur.execute('''SELECT species, COUNT(*) FROM protein GROUP BY species;''')
x = PrettyTable([headers[0] for headers in cursor.description])
for row in cursor.fetchall():
    x.add_row(row)
print(x)

+---------+----------+
| species | COUNT(*) |
+---------+----------+
|    Br   |    6     |
|    Bt   |    6     |
|    Ce   |    50    |
|    Dm   |    16    |
|    Gg   |    11    |
|    Hs   |    47    |
|    Mm   |    42    |
|    Tr   |    1     |
|    Xl   |    16    |
+---------+----------+


In [11]:
print ("7.Adding structure information:")
#7.create two more tables, for storing structures and their owners
c.executescript('''
CREATE TABLE IF NOT EXISTS structure (
    name character varying(100) PRIMARY KEY,
    resolution character varying(10),
    method character varying(50)
);
CREATE TABLE IF NOT EXISTS proteinmembers (
    protein character varying(20),
    structure character varying(100),
    PRIMARY KEY(protein,structure)
);
'''
)
c.execute("SELECT sql FROM sqlite_master WHERE type='table';")
col_headers = [t[0] for t in c.description]
printdb(c.fetchall(),col_headers)

7.Adding structure information:


NameError: name 'c' is not defined